In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.datasets import load_nfl
from yellowbrick.cluster import KElbowVisualizer
from itertools import permutations

In [ ]:
# Read in the data
df_firm_data = pd.read_csv('../Daten/FDS/Firm_Data.csv', index_col=0)
df_id_data = pd.read_csv('../Daten/FDS/ID_Data.csv', index_col=0)
df_mc_data = pd.read_csv('../Daten/FDS/MC_Data.csv', index_col=0)
df_return_data = pd.read_csv('../Daten/FDS/Return_Data.csv', index_col=0)

In [ ]:
def show_data(df_firm_data, df_id_data, df_mc_data, df_return_data):
    #show heads if true
    print("firm data")
    display(df_firm_data.head())
    print("id data")
    display(df_id_data.head())
    print("mc data (market cap)")
    display(df_mc_data.head())
    print("return data")
    display(df_return_data.head())


In [ ]:
#make the dataframes a little bit more readable
def make_readable(df_firm_data, df_id_data, df_mc_data, df_return_data):
    """- sets date as index where suitaeble"""
    df_mc_data.index = pd.to_datetime(df_mc_data["Date"])
    df_return_data.index = pd.to_datetime(df_return_data["Date"])
    #drop date column
    df_mc_data.drop("Date", axis=1, inplace=True)
    df_return_data.drop("Date", axis=1, inplace=True)
    #sort index
    df_mc_data.sort_index(inplace=True)
    df_return_data.sort_index(inplace=True)
    return df_firm_data, df_id_data, df_mc_data, df_return_data
    
df_firm_data, df_id_data, df_mc_data, df_return_data = make_readable(df_firm_data, df_id_data, df_mc_data, df_return_data)

In [41]:
show_data(df_firm_data, df_id_data, df_mc_data, df_return_data)

firm data


,Date,Total Current Assets,Total Current Liabilities,Total Debt,"Total Assets, Reported",Net Income - Actual,Revenue Per Share,Total Revenue,Total Equity,Total CO2 Equivalent Emissions To Revenues USD in million,ESG Score,Social Pillar Score,Governance Pillar Score,Environmental Pillar Score,Company Market Capitalization,"Property Plant And Equipment, Total - Gross",P/E (Daily Time Series Ratio)
RIC,,,,,,,,,,,,,,,,,
BVIC.L,31/12/2009,434.373405,483.824655,718.957376,1361.504594,107.309409,7.068281,1561.383359,-3.988004,NaN,51.267135,45.539754,59.077651,53.535053,1430.561330,817.381317,19.299905
BVIC.L,31/12/2010,579.778906,580.095207,901.298414,1655.358922,139.656087,8.006665,1800.699024,-48.552134,NaN,50.550242,57.622888,31.890959,55.261467,1772.376566,889.120843,16.927900
BVIC.L,31/12/2011,598.968478,607.694345,893.154869,1660.096296,122.255528,7.912984,2010.689188,35.059289,NaN,46.732870,57.397993,28.924942,45.229496,1205.126197,836.280911,13.534436
BVIC.L,31/12/2012,615.523874,601.137943,907.121844,1658.099764,100.765756,7.914423,2030.840850,59.968319,NaN,57.941343,70.633040,37.216902,55.762859,1603.272359,872.046035,18.363571
BVIC.L,31/12/2013,748.229068,814.064189,895.874010,1714.940377,133.919162,8.716901,2133.025672,65.996482,31.296503,49.513243,52.778937,33.146107,58.148148,2819.919676,907.653333,27.421399


id data


,RIC,ISIN,Name,Sector,Country
0,INTU.O,US4612021034,Intuit Inc,Technology,USA
1,HL,US4227041062,Hecla Mining Co,Basic Materials,USA
2,GILG.DE,DE0005878003,DMG Mori AG,Industrials,Germany
3,APH,US0320951017,Amphenol Corp,Technology,USA
4,SGH.O,KYG8232Y1017,Smart Global Holdings Inc,Technology,USA


mc data (market cap)


,INTU.O,HL,GILG.DE,APH,SGH.O,PMAG.S,RTX,SOLARb.CO,OBEL.BR,WIZZ.L,...,XYL,SCHN.PA,PRGO.K,TRELb.ST,SRG.MI,LOG.MC,GEO.MI,NRG,MYTr.AT,TEMN.S
Date,,,,,,,,,,,,,,,,,,,,,
2005-12-31,9431.218016,481.369276,301.982906,3933.733862,0.0,42.394244,56952.86267,137.630413,4959.476223,0.0,...,0.0,20180.11060,1392.772801,1726.035361,8042.219005,0.0,2844.375572,3802.640450,875.088506,536.676851
2006-01-31,9259.580465,626.017186,360.516482,4539.726825,0.0,42.389265,59270.48035,136.699953,4707.277568,0.0,...,0.0,23643.19825,1452.563121,1727.326920,8476.565811,0.0,2939.943338,3895.446827,1242.051819,573.175661
2006-02-28,8463.455941,590.448028,355.606967,4486.139650,0.0,42.058317,59402.48588,133.325876,4445.806575,0.0,...,0.0,23118.43272,1478.618065,1803.951905,8589.988426,0.0,3168.139495,3490.326814,1237.343398,590.161549
2006-03-31,9260.876805,784.553756,380.403430,4660.297968,0.0,43.129885,58600.03703,144.741545,4592.497777,0.0,...,0.0,24466.27625,1517.700481,2015.665077,8642.636110,0.0,3450.354232,6194.021936,1142.026571,541.855421
2006-04-30,9431.503977,767.936884,437.669801,5162.231172,0.0,46.655356,63492.63974,648.915358,5088.864262,0.0,...,0.0,25696.54916,1500.917885,1930.076236,8779.218311,0.0,3698.005018,6518.653337,1185.646517,545.485527


return data


,INTU.O,HL,GILG.DE,APH,SGH.O,PMAG.S,RTX,SOLARb.CO,OBEL.BR,WIZZ.L,...,XYL,SCHN.PA,PRGO.K,TRELb.ST,SRG.MI,LOG.MC,GEO.MI,NRG,MYTr.AT,TEMN.S
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,0.625652,-12.000000,11.671470,1.971831,0.0,0.0,-18.557692,0.000000,-25.558487,0.0,...,0.0,-6.991661,7.421875,-1.045752e+01,0.0,0.0,0.0,0.0,-23.040752,0.0
2000-02-29,-12.953368,4.545455,-2.580645,17.771639,0.0,0.0,-3.408844,-2.777778,13.397921,0.0,...,0.0,-7.586207,-14.545455,-1.021898e+01,0.0,0.0,0.0,0.0,-18.533605,0.0
2000-03-31,3.571429,-4.347826,3.973510,27.912432,0.0,0.0,24.049080,3.428571,-1.521150,0.0,...,0.0,-0.447761,3.404255,-8.130081e-01,0.0,0.0,0.0,0.0,1.125000,0.0
2000-04-30,-33.908046,-9.090909,5.732484,24.694377,0.0,0.0,-1.582591,2.287293,-14.621244,0.0,...,0.0,7.946027,-25.102881,3.516658e+00,0.0,0.0,0.0,0.0,-20.519159,0.0
2000-05-31,0.869565,-15.000000,11.862429,-28.235294,0.0,0.0,-2.515897,2.040297,-9.491945,0.0,...,0.0,-0.833333,5.494505,-4.023120e-10,0.0,0.0,0.0,0.0,19.284603,0.0


In [106]:
def make_df_visualize_without_clustering(df_firm_data, df_id_data, df_mc_data, df_return_data, visualize=True,
                                         show_time=False, color_what = "Sector"):
    """
    - makes a dataframe for visualization without clustering
    - if visualize is true, it will show the data
    - it visualizes calculates the return, risk, and esg score
    - the colorization can by set, by default it is set to the industry
    - the company name of a datapoint is shown when hovering over it
    - the first visualization is a 3D scatterplot with the avg return, avg risk, and avg esg score, and the 
    colorization is set to the Sector by default, can be changed with the color_what parameter
    - it is also possible to visualize the data in 3d with the axes ESG, Return, and time if show_time is set to true
    - if plot hyperplane is True it will also plot the hyperplane with the "best" companies
    """
    #base data
    df_company = df_id_data.copy()
    #calculate the average return in a dictionary (use arethmetic mean)
    dict_return_mean = {}
    dict_risk_mean = {}
    dict_esg_mean = {}

    for column in df_return_data.columns:
        if column != "Date":
            dict_return_mean[column] = df_return_data[column].mean()
            dict_risk_mean[column] = df_return_data[column].std()
            dict_esg_mean[column] = df_firm_data[df_firm_data.index == column]["ESG Score"].mean()

    #add the data to the dataframe
    df_return_esg_time = df_firm_data.copy()
    #remove all columns except the date, esg score, and return
    df_return_esg_time.drop(df_return_esg_time.columns.difference(["Date", "ESG Score", "Return"]), 1, inplace=True)    
    #remove rows with nan values
    df_return_esg_time.dropna(inplace=True)
    #save indes as column
    df_return_esg_time["Company"] = df_return_esg_time.index
    
    #get value baes on index and date from df_return_data
    def get_value(index, date):

        try:
            return df_return_data.loc[date][index]
        except:
            return np.nan
    #append returns based on the index and the date
    df_return_esg_time['return'] = df_return_esg_time.apply(lambda x: get_value(index = x["Company"], date = x['Date']), axis=1)
    #remove extreme outliers
    df_company = df_company[df_company["avg_return"] < 3*df_company["avg_return"].median()]
    
    #create a new dataframe with the return and esg score on a daily basis
    df_return_esg = df_return_data.copy()
 

    
    #visualize the data
    if visualize:
        #make the 3d scatterplot
        fig = px.scatter_3d(df_company, x="avg_return", y="avg_risk", z="avg_esg", color=color_what,
                            hover_name="Name")
        #make smaller points
        fig.update_traces(marker=dict(size=4))
        #add hyperplane, dont use go since it is not working

        
        fig.show()
        #make the 3d scatterplot with time
        if show_time:
            fig = px.scatter_3d(df_company, x="", y="avg_risk", z="avg_esg", color=color_what, 
                                hover_name="Company Name", animation_frame="Date")
            fig.show()
    return df_company
    
    

In [107]:
make_df_visualize_without_clustering(df_firm_data, df_id_data, df_mc_data, df_return_data,color_what = "Country", visualize=True, show_time=True)

KeyError: 'avg_return'